# Delta Lake Rollback with RESTORE

In [1]:
import pyspark
from delta import *

In [2]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .config("spark.databricks.delta.retentionDurationCheck.enabled", "false")
)

In [3]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/matthew.powers/opt/miniconda3/envs/pyspark-322-delta-200/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/matthew.powers/.ivy2/cache
The jars for the packages stored in: /Users/matthew.powers/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1ff052dc-a1cb-41d5-bdef-7c7600640a7c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.0.0 in central
	found io.delta#delta-storage;2.0.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 347ms :: artifacts dl 24ms
	:: modules in use:
	io.delta#delta-core_2.12;2.0.0 from central in [default]
	io.delta#delta-storage;2.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number|

## Create Delta Lake table

In [4]:
df = spark.range(0, 3)

In [5]:
df.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
+---+



In [6]:
df.write.format("delta").save("tmp/delta-table")

In [7]:
df2 = spark.range(4, 6)

In [8]:
df2.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [9]:
df2.show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [10]:
df3 = spark.range(7, 10)

In [25]:
df3.show()

+---+
| id|
+---+
|  7|
|  8|
|  9|
+---+



In [11]:
df3.write.mode("overwrite").format("delta").save("tmp/delta-table")

In [12]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  7|
|  9|
|  8|
+---+



## Time travel to different versions of Delta Lake

In [13]:
spark.read.format("delta").option("versionAsOf", "0").load("tmp/delta-table").show()

+---+
| id|
+---+
|  0|
|  2|
|  1|
+---+



In [14]:
spark.read.format("delta").option("versionAsOf", "1").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [15]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

+---+
| id|
+---+
|  7|
|  9|
|  8|
+---+



## Restore v1 of Delta Lake

In [16]:
from delta.tables import *

In [17]:
deltaTable = DeltaTable.forPath(spark, "tmp/delta-table")

In [18]:
deltaTable.restoreToVersion(1)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [19]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [20]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

+---+
| id|
+---+
|  7|
|  9|
|  8|
+---+



## Vacuum Delta Lake

In [21]:
# Two different ways to vacuum
deltaTable.vacuum(0)
# spark.sql("VACUUM 'tmp/delta-table'")

Deleted 8 files and directories in a total of 1 directories.


DataFrame[]

In [22]:
spark.read.format("delta").load("tmp/delta-table").show()

+---+
| id|
+---+
|  4|
|  5|
+---+



In [23]:
spark.read.format("delta").option("versionAsOf", "2").load("tmp/delta-table").show()

22/08/28 19:11:24 ERROR Executor: Exception in task 0.0 in stage 179.0 (TID 12128)
java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00003-f21432cc-38bc-4e50-a7f7-57a5d82abd62-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)


Py4JJavaError: An error occurred while calling o100.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 179.0 failed 1 times, most recent failure: Lost task 0.0 in stage 179.0 (TID 12128) (qtk9h72yp0 executor driver): java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00003-f21432cc-38bc-4e50-a7f7-57a5d82abd62-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.

## Cannot time travel back after the vacuum

In [24]:
spark.read.format("delta").option("versionAsOf", "0").load("tmp/delta-table").show()

22/08/28 19:11:50 ERROR Executor: Exception in task 0.0 in stage 188.0 (TID 12231)
java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00003-367d5d44-9e72-4e94-822e-891f05001843-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:187)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:104)


Py4JJavaError: An error occurred while calling o107.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 188.0 failed 1 times, most recent failure: Lost task 0.0 in stage 188.0 (TID 12231) (qtk9h72yp0 executor driver): java.io.FileNotFoundException: 
File file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/delta-table/part-00003-367d5d44-9e72-4e94-822e-891f05001843-c000.snappy.parquet does not exist

It is possible the underlying files have been updated. You can explicitly invalidate
the cache in Spark by running 'REFRESH TABLE tableName' command in SQL or by
recreating the Dataset/DataFrame involved.
       
	at org.apache.spark.sql.errors.QueryExecutionErrors$.readCurrentFileNotFoundError(QueryExecutionErrors.scala:506)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.org$apache$spark$sql$execution$datasources$FileScanRDD$$anon$$readCurrentFile(FileScanRDD.scala:130)
	at org.apache.spark.

## Cleanup

In [70]:
%rm -rf tmp